# State Management and Tool Integration

## Introduction

In this notebook, we'll explore:
- Advanced state management patterns
- Integrating external tools with agents
- Building agents with memory
- Checkpointing and persistence

These are essential for building production-ready AI agents.

## Part 1: Advanced State Management

### Complex State Schemas

Real agents need to track multiple pieces of information:

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
import operator

class ComplexAgentState(TypedDict):
    # Conversation history
    messages: Annotated[Sequence[BaseMessage], operator.add]
    
    # Agent's internal thoughts
    thoughts: Annotated[list[str], operator.add]
    
    # Tools used and their outputs
    tool_history: Annotated[list[dict], operator.add]
    
    # Current task decomposition
    current_plan: list[str]
    
    # Metadata
    iteration: int
    max_iterations: int
    
    # Error tracking
    errors: Annotated[list[str], operator.add]
    
    # Final output
    final_answer: str | None

### State Reducers

Custom logic for how state updates are merged:

In [ ]:
from typing import Any

def merge_unique_items(existing: list, new: list) -> list:
    """Merge lists while keeping unique items"""
    combined = existing + new
    return list(dict.fromkeys(combined))  # Remove duplicates

def merge_dicts_deep(existing: dict, new: dict) -> dict:
    """Deep merge dictionaries"""
    result = existing.copy()
    for key, value in new.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = merge_dicts_deep(result[key], value)
        else:
            result[key] = value
    return result

class SmartState(TypedDict):
    tags: Annotated[list, merge_unique_items]
    metadata: Annotated[dict, merge_dicts_deep]
    counter: int  # Default: replace

## Part 2: Tool Integration

### Defining Tools

Tools are functions that agents can call to interact with the world:

In [ ]:
from langchain_core.tools import tool
from datetime import datetime
import requests

@tool
def get_current_time() -> str:
    """Get the current time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def search_web(query: str) -> str:
    """Search the web for information.
    
    Args:
        query: The search query
    """
    # Simulated web search
    return f"Search results for '{query}': [Result 1, Result 2, Result 3]"

@tool
def calculate(expression: str) -> str:
    """Safely evaluate a mathematical expression.
    
    Args:
        expression: Math expression like '2 + 2' or '10 * 5'
    """
    try:
        # Safe evaluation (in production, use a proper parser)
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def read_file(filepath: str) -> str:
    """Read contents of a file.
    
    Args:
        filepath: Path to the file
    """
    try:
        with open(filepath, 'r') as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

# Collect all tools
tools = [get_current_time, search_web, calculate, read_file]

### Building a Tool-Using Agent

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 1. Define state
class ToolAgentState(TypedDict):
    messages: Annotated[list, operator.add]
    iterations: int

# 2. Initialize LLM with tools
llm = ChatOpenAI(model="gpt-4", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# 3. Tool executor
tool_executor = ToolExecutor(tools)

# 4. Define agent node
def call_agent(state: ToolAgentState) -> ToolAgentState:
    """Agent decides what to do next"""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {
        "messages": [response],
        "iterations": state["iterations"] + 1
    }

# 5. Define tool execution node
def execute_tools(state: ToolAgentState) -> ToolAgentState:
    """Execute tools requested by the agent"""
    last_message = state["messages"][-1]
    
    # Execute each tool call
    tool_messages = []
    for tool_call in last_message.tool_calls:
        result = tool_executor.invoke(tool_call)
        tool_messages.append(
            ToolMessage(
                content=str(result),
                tool_call_id=tool_call["id"]
            )
        )
    
    return {"messages": tool_messages, "iterations": state["iterations"]}

# 6. Define routing
def should_continue(state: ToolAgentState) -> str:
    """Determine next step"""
    last_message = state["messages"][-1]
    
    # Check iteration limit
    if state["iterations"] >= 10:
        return "end"
    
    # If agent called tools, execute them
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "continue"
    
    # Otherwise, we're done
    return "end"

# 7. Build graph
workflow = StateGraph(ToolAgentState)

workflow.add_node("agent", call_agent)
workflow.add_node("tools", execute_tools)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "end": END
    }
)

workflow.add_edge("tools", "agent")

# 8. Compile
agent_app = workflow.compile()

# 9. Test the agent
print("Tool-Using Agent Example")
print("=" * 50)

result = agent_app.invoke({
    "messages": [HumanMessage(content="What time is it? Then calculate 15 * 24")],
    "iterations": 0
})

print("\nFinal Response:")
print(result["messages"][-1].content)

### Tool Flow Visualization

```
User: "What time is it? Then calculate 15 * 24"
   │
   ▼
┌──────────┐
│  Agent   │ → Decides to call get_current_time()
└────┬─────┘
     │
     ▼
┌──────────┐
│  Tools   │ → Executes: "2024-01-15 14:30:00"
└────┬─────┘
     │
     ▼
┌──────────┐
│  Agent   │ → Decides to call calculate("15 * 24")
└────┬─────┘
     │
     ▼
┌──────────┐
│  Tools   │ → Executes: "360"
└────┬─────┘
     │
     ▼
┌──────────┐
│  Agent   │ → Formulates final answer
└────┬─────┘
     │
     ▼
   END
```

## Part 3: Persistence and Checkpointing

### Why Persistence?

Persistence allows agents to:
- Resume interrupted conversations
- Maintain long-term memory
- Recover from failures
- Support human-in-the-loop workflows

### Using Memory Checkpointer

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Create memory checkpointer
memory = MemorySaver()

# Compile graph with checkpointing
persistent_agent = workflow.compile(checkpointer=memory)

# Configuration for this conversation
config = {"configurable": {"thread_id": "conversation-1"}}

# First interaction
print("First interaction:")
result1 = persistent_agent.invoke(
    {"messages": [HumanMessage("My name is Alice")], "iterations": 0},
    config
)
print(result1["messages"][-1].content)

# Second interaction (agent remembers context)
print("\nSecond interaction (in same thread):")
result2 = persistent_agent.invoke(
    {"messages": [HumanMessage("What's my name?")], "iterations": 0},
    config
)
print(result2["messages"][-1].content)

### SQLite Checkpointer (Production)

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

# Create SQLite checkpointer
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    # Compile with persistence
    persistent_app = workflow.compile(checkpointer=checkpointer)
    
    # Use the agent
    config = {"configurable": {"thread_id": "user-123"}}
    result = persistent_app.invoke(
        {"messages": [HumanMessage("Hello!")], "iterations": 0},
        config
    )

## Part 4: Human-in-the-Loop

### Interrupt for Approval

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

class HumanLoopState(TypedDict):
    request: str
    approved: bool | None
    result: str

def make_request(state: HumanLoopState) -> HumanLoopState:
    """Agent makes a request that needs approval"""
    state["request"] = "I want to delete all files"
    return state

def execute_if_approved(state: HumanLoopState) -> HumanLoopState:
    """Execute only if approved"""
    if state["approved"]:
        state["result"] = "Files deleted"
    else:
        state["result"] = "Action cancelled"
    return state

# Build graph
workflow = StateGraph(HumanLoopState)
workflow.add_node("request", make_request)
workflow.add_node("execute", execute_if_approved)

workflow.set_entry_point("request")
workflow.add_edge("request", "execute")
workflow.add_edge("execute", END)

# Compile with interrupt
memory = MemorySaver()
app_with_human = workflow.compile(
    checkpointer=memory,
    interrupt_before=["execute"]  # Pause before execute
)

# Run and pause
config = {"configurable": {"thread_id": "1"}}
result = app_with_human.invoke(
    {"request": "", "approved": None, "result": ""},
    config
)

print(f"Agent requests: {result['request']}")
print("Waiting for human approval...")

# Human approves/rejects
result["approved"] = False  # Human rejects

# Resume execution
final = app_with_human.invoke(result, config)
print(f"Result: {final['result']}")

## Part 5: Memory Strategies

### Short-term vs Long-term Memory

In [ ]:
class MemoryState(TypedDict):
    # Short-term: Recent conversation (limited)
    recent_messages: Annotated[list, lambda x, y: (x + y)[-10:]]  # Keep last 10
    
    # Long-term: Important facts (unlimited)
    facts: Annotated[dict, merge_dicts_deep]
    
    # Working memory: Current task context
    working_context: str

def extract_facts(state: MemoryState) -> MemoryState:
    """Extract important information to long-term memory"""
    # Use LLM to identify facts from recent messages
    new_facts = {
        "user_name": "Alice",
        "preferences": {"language": "Python"}
    }
    state["facts"] = new_facts
    return state

### Vector Store Memory (for Semantic Search)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

class VectorMemoryState(TypedDict):
    messages: Annotated[list, operator.add]
    vector_store: Any  # FAISS vector store
    relevant_memories: list[str]

def retrieve_relevant_memories(state: VectorMemoryState) -> VectorMemoryState:
    """Search vector store for relevant past interactions"""
    current_query = state["messages"][-1].content
    
    # Search for similar past conversations
    if state["vector_store"]:
        results = state["vector_store"].similarity_search(current_query, k=3)
        state["relevant_memories"] = [doc.page_content for doc in results]
    
    return state

def store_interaction(state: VectorMemoryState) -> VectorMemoryState:
    """Add current interaction to vector store"""
    last_message = state["messages"][-1].content
    
    if state["vector_store"]:
        state["vector_store"].add_texts([last_message])
    
    return state

## Exercises

### Exercise 1: Build a Calculator Agent
Create an agent that:
1. Can solve multi-step math problems
2. Uses the calculate tool
3. Shows its reasoning

### Exercise 2: Add Error Handling
Extend an agent to:
1. Catch and handle tool errors
2. Retry failed operations
3. Report errors to the user

### Exercise 3: Build a Research Agent
Create an agent that:
1. Searches for information
2. Synthesizes multiple sources
3. Cites its sources

### Exercise 4: Implement Conversation Memory
Build an agent that:
1. Remembers user preferences
2. Refers to earlier conversation
3. Persists across sessions

## Key Takeaways

- **Complex state** requires careful schema design and reducers
- **Tools** extend agent capabilities to interact with the world
- **Persistence** enables long-running conversations and recovery
- **Human-in-the-loop** adds safety and control
- **Memory strategies** balance context vs. performance

## Next Steps

In the next notebook:
- Advanced agent patterns (ReAct, Plan-Execute, Reflection)
- Multi-agent systems
- Production optimization

---

**Next**: [04_advanced_patterns.ipynb](./04_advanced_patterns.ipynb)